# CUDA 
* pip install cudatoolkit
* Usual cuda operation in C/C++
    * Move data from host memory from host to device memory.
    * Launch the kernel with correct grid dimension.
    * Execute kernel on the device.
    * Move data from device memory to host memory.

In [7]:
import numba
from numba import jit, int32, prange, vectorize, float64, cuda
import numpy as np
import math


## Kernel Function and Device Function
In Numba, a Just-In-Time (JIT) compiler for Python that supports CUDA programming, the terms **kernel** and **device function** refer to different types of GPU functions that can be executed on CUDA-capable devices.

### 1. **Kernel Function**

- **Purpose**: Kernel functions are entry points to launch computations on the GPU. They are the top-level functions that are called from the host (CPU) and executed on the device (GPU).
- **Execution**: A kernel function is executed by many parallel threads on the GPU, which are organized in grids and blocks.
- **Calling**: Kernel functions are invoked from the host using a special syntax specifying the grid and block dimensions. In Numba, this is done using triple brackets `<<<grid_size, block_size>>>`.
- **Return Value**: Kernel functions cannot return a value; instead, they work by modifying the input/output arrays passed as arguments.
  

### 2. **Device Function**

- **Purpose**: Device functions are helper functions that run on the GPU and can only be called from other GPU functions (such as other device functions or kernels). They provide code reuse within the GPU.
- **Execution**: Device functions are executed on the GPU as part of a kernel or another device function. They are not directly callable from the host (CPU).
- **Calling**: Device functions are called like regular Python functions, but only from within other GPU functions.
- **Return Value**: Unlike kernels, device functions can return values, making them useful for computations that need to be reused within the GPU code.
  

### Key Differences:

| **Aspect**            | **Kernel Function**                                | **Device Function**                                   |
|-----------------------|----------------------------------------------------|-------------------------------------------------------|
| **Callable From**      | Host (CPU)                                         | GPU (inside another kernel or device function)        |
| **Return Value**       | No return values (modifies arguments)              | Can return values                                     |
| **Launch Syntax**      | Launched with `<<<grid_size, block_size>>>` syntax | Called like a regular Python function                 |
| **Purpose**            | Entry point for GPU computation                    | Reusable helper functions within GPU code             |
| **Execution Context**  | Runs on the GPU, callable from the host            | Runs on the GPU, callable only from another GPU function|

In summary, **kernel functions** are the primary interface for launching GPU computations from the CPU, while **device functions** are helper functions that run on the GPU and can be reused in kernels or other device functions.

Numba automates the following:

* Allcated GPU memory.
* Copy data to the GPU memory.
* Executed the CUDA kernel with the *correct kernel dimensions given the input sizes*.
* Copy data to the host memory.
* Return the result as a NumPy array. 

* We can also create normal function that are called from a vectorized fucntion
* Using cuda.jit

In [35]:
@cuda.jit(device=True) #This function will only be executed on a GPU
def polar_to_cartesian(rho, theta):
    x = rho * math.cos(theta)
    y = rho * math.sin(theta)
    return x, y  

@vectorize(['float32(float32, float32, float32, float32)'], target='cuda')
def polar_distance(rho1, theta1, rho2, theta2):
    x1, y1 = polar_to_cartesian(rho1, theta1)
    x2, y2 = polar_to_cartesian(rho2, theta2)
    
    return ((x1 - x2)**2 + (y1 - y2)**2)**0.5


In [36]:
x = np.arange(100, dtype='float32').reshape(1, 100)
polar_distance(x, x, x, x)

/scratch/vp91/Training-Venv/intro-parallel-prog/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.2013700e-07,
        0.0000000e+00, 2.9802322e-08, 0.0000000e+00, 2.1490760e-07,
        0.0000000e+00, 1.8848644e-07, 1.6858739e-07, 2.3842040e-07,
        0.0000000e+00, 2.7476383e-07, 2.3841858e-07, 5.3312016e-07,
        0.0000000e+00, 6.7959752e-07, 5.9604645e-07, 7.7843413e-07,
        2.3841858e-07, 9.5553514e-07, 5.9604935e-07, 4.2146849e-07,
        4.7683716e-07, 4.9151248e-07, 1.1920929e-06, 4.2981520e-07,
        3.3717478e-07, 5.1273855e-07, 5.3312016e-07, 8.5340866e-07,
        0.0000000e+00, 1.4901397e-06, 1.1801118e-06, 2.5288108e-07,
        1.9110703e-06, 8.3659103e-07, 1.7970578e-06, 1.1876141e-06,
        4.7683716e-07, 1.6801348e-06, 9.5367432e-07, 5.6230908e-07,
        1.6690798e-06, 1.2287812e-06, 1.1980385e-06, 1.7377591e-06,
        9.5367432e-07, 1.6858739e-07, 6.3360608e-07, 1.7721735e-06,
        3.5762787e-07, 1.5557270e-06, 2.6656008e-07, 1.1921060e-06,
        1.3486991e-06, 9.5040912e-07, 1.3134171e


## Thread Indexing

In [35]:
@cuda.jit
def increment_a_2D_array(an_array):
    x = cuda.threadIdx.x
    y = cuda.threadIdx.y

    # or x, y = cuda.grid(2)
    
    if x < an_array.shape[0] and y < an_array.shape[1]:
        an_array[x, y] += 1

an_array = np.random.rand(1000000)
an_array = an_array.reshape(1000, 1000)

threadsperblock = (16, 16)
blockspergrid_x = math.ceil(an_array.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(an_array.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

increment_a_2D_array[blockspergrid, threadsperblock](an_array)

/scratch/vp91/Training-Venv/intro-parallel-prog/lib/python3.11/site-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


## Memory Management
Data management is automatic in Numba, but we can also manage it manually.

### Host to device copy

In [ ]:
data_cpu = np.arange(10)
data_gpu = cuda.to_device(data_cpu)

### Device to host copy

In [13]:
data_cpu = data_gpu.copy_to_host()

In [14]:
data_gpu.copy_to_host(data_cpu)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

## Streams

In [15]:
@cuda.jit
def add_kernel(a, b, c):
    tx = cuda.threadIdx.x
    ty = cuda.blockIdx.x
    bw = cuda.blockDim.x

    pos = tx + ty * bw

    if pos < a.size:
        c[pos] = a[pos] + b[pos]

In [16]:
# Create two streams
stream1 = cuda.stream()
stream2 = cuda.stream()

In [17]:
# Initialize data
size = 1000000
a_cpu = np.arange(size, dtype=np.float32)
b_cpu = np.arange(size, dtype=np.float32) * 2
c_cpu = np.zeros(size, dtype=np.float32)

In [18]:
# Allocate device memory
a_gpu = cuda.to_device(a_cpu)
b_gpu = cuda.to_device(b_cpu)
c_gpu = cuda.device_array(size, dtype=np.float32)

In [19]:
# Define block and grid dimensions
threads_per_block = 256
blocks_per_grid = (size + (threads_per_block - 1)) // threads_per_block

In [21]:
# Launch kernels in different streams
add_kernel[blocks_per_grid, threads_per_block, stream1](a_gpu, b_gpu, c_gpu)
add_kernel[blocks_per_grid, threads_per_block, stream2](b_gpu, c_gpu, a_gpu)

In [22]:
# Wait for the streams to complete
stream1.synchronize()
stream2.synchronize()

In [23]:
# Copy result back to host
c_cpu = c_gpu.copy_to_host()

## Vectorization in GPU

(n),()->(n) tells NumPy that the function takes a n-element one-dimension array, a scalar, denoted by the empty tuple (), and computes an n-element one-dimension array.

Unlike vectorize() functions, guvectorize() functions should not return any result

In [25]:
from numba import guvectorize, int64

@guvectorize([(int64[:], int64, int64[:])], '(n),()->(n)')
def g(x, y, res):
    for i in range(x.shape[0]):
        res[i] = x[i] + y

In [26]:

x = np.arange(size, dtype=np.int64)
y = 10  # Scalar value

# Invoke the function
result = g(x, y)

In [27]:
print(result)

[     10      11      12 ... 1000007 1000008 1000009]


In Numba's `@guvectorize` functions, there is no explicit return statement. Instead, the output is passed via the output argument (in this case, `res`). Numba modifies this array in place. When invoking the function, the result is automatically returned because Numba allocates an output array for you.


### Key Points:
1. **In-place Modification**: The `res` array is the output, which is modified in place within the `guvectorize` function.
2. **Return**: When calling the `guvectorize`-decorated function, even though the function doesn't explicitly return anything, Numba provides the output array based on the function signature.



### How the Return Works:

- **`result = g(x, y)`**: Numba handles the allocation of the `res` array internally and returns it automatically after the function finishes.
- You don't need to declare or pre-allocate the `res` array when calling the function; Numba will do this for you.
  
Thus, the array `result` contains the values produced by the in-place modification of `res` inside the `g` function.


## Reduction in GPU

In [29]:
@cuda.reduce
def sum_reduce(a, b):
    return a + b

In [34]:
size = 1000000
A = np.arange(size, dtype=np.int64)
normal_sum = A.sum()      # NumPy sum reduction
gpu_sum = sum_reduce(A)   # cuda sum reduction
assert normal_sum == gpu_sum

/scratch/vp91/Training-Venv/intro-parallel-prog/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/scratch/vp91/Training-Venv/intro-parallel-prog/lib/python3.11/site-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
/scratch/vp91/Training-Venv/intro-parallel-prog/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


### What happens if you change the type to np.float32

## Excercise 3
[Go to Notebook Exercise 3](./Exercise3.ipynb)